In [1]:
# Mounting drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
! unzip gdrive/My\ Drive/fer2013.zip

Archive:  gdrive/My Drive/fer2013.zip
  inflating: fer2013.csv             


In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
num_classes = 7 # angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5

In [0]:
# with open("fer2013.csv") as f:
#   content = f.readlines()
 
# lines = np.array(content)
 
# num_of_instances = lines.size
# print("number of instances: ",num_of_instances)

In [0]:
# for i in range(1, num_of_instances):
  
#   try:
#     emotion, img, usage = lines[i].split(",")
    
#     val = img.split(" ")
#     pixels = np.array(val, 'float32')
    
#     if 'Training' in usage:
#       y_train.append(emotion)
#       x_train.append(pixels)
      
#     elif 'PublicTest' in usage:
#       y_test.append(emotion)
#       x_test.append(pixels)
      
#   except:
#     print("", end="")

In [7]:
import pandas as pd
data = pd.read_csv('fer2013.csv', header=0)
data.columns

Index(['emotion', 'pixels', 'Usage'], dtype='object')

In [10]:
grouped = data.groupby('Usage')

train = grouped.get_group('Training')
test = grouped.get_group('PrivateTest')
validation = grouped.get_group('PublicTest')
print(len(train), len(test), len(validation))

28709 3589 3589


In [0]:
df = pd.DataFrame(train)
df.drop(['Usage'], axis=1, inplace=True)
df.to_csv("train.csv", index=False)

# df = pd.DataFrame(y_train)
# df.to_csv("y_train.csv", index=False)

df = pd.DataFrame(test)
df.drop(['Usage'], axis=1, inplace=True)
df.to_csv("test.csv", index=False)

# df = pd.DataFrame(y_test)
# df.to_csv("y_test.csv", index=False)

df = pd.DataFrame(validation)
df.drop(['Usage'], axis=1, inplace=True)
df.to_csv("validation.csv", index=False)

In [12]:
! pip install torch torchvision

In [0]:
import torch.utils.data as Data

In [0]:
class FerDataset(Data.Dataset):
  
  def __init__(self, csv_path, transforms=None, train=False, valid=False):
    
    self.transform = transforms
    
    #self.split = split
    
    self.train = train
    
    self.valid = valid
    
    #self.data = pd.read_csv(csv_path)
    
    
    with open(csv_path) as f:
      content = f.readlines()
 
    lines = np.array(content)
 
    num_of_instances = lines.size
    #print("number of instances: ",num_of_instances)
    
    self.train_data, self.train_labels, self.Test_data, self.Test_labels, self.valid_data, self.valid_labels = [], [], [], [], [], []
    
    for i in range(1, num_of_instances):
  
      try:
        emotion, img = lines[i].split(",")
        
        val = img.split(" ")
        pixels = np.array(val, 'int64')
      
        if self.train == True:
          self.train_labels.append(emotion)
          self.train_data.append(pixels)
          
        elif self.valid == True:
          self.valid_labels.append(emotion)
          self.valid_data.append(pixels)
      
        else:
          self.Test_labels.append(emotion)
          self.Test_data.append(pixels)
      
      except:
        print("", end="")
    
    
    
    
    
    
    if self.train == True:
      
#       self.train_data = self.data['pixels']
#       self.train_labels = self.data['emotion']
       self.train_data = np.asarray(self.train_data)
#       print(type(self.train_data))
#       print(type(self.train_data[1]))
       self.train_data = self.train_data.reshape((28709, 48, 48))
#       print(self.train_data[1])
     
    
    elif self.valid == True: 
      
#       self.Test_data = self.data['pixels']
#       self.Test_labels = self.data['emotion']
       self.valid_data = np.asarray(self.valid_data)
       self.valid_data = self.valid_data.reshape((3589, 48, 48))
    
    else: 
      
#       self.Test_data = self.data['pixels']
#       self.Test_labels = self.data['emotion']
       self.Test_data = np.asarray(self.Test_data)
       self.Test_data = self.Test_data.reshape((3589, 48, 48))
      
      
  def __len__(self):
    
    if self.train == True:
      return len(self.train_data)
    
    elif self.valid == True:
      return len(self.valid_data)
    
    else:
      return len(self.Test_data)
    
  
  
  def __getitem__(self):
    
    if self.train == True:
      
      img, target = self.train_data, self.train_labels
      
    elif self.valid == True:
      
      img, target = self.valid_data, self.valid_labels

    else:
      
      img, target = self.Test_data, self.Test_labels
      
#     img = img[:, :, np.newaxis]
#     img = np.concatenate((img, img, img), axis=2)
#     img = Image.fromarray(img)
    
    if self.transform is not None:
      
      img = self.transform(img)
        
    return img, target
    
      
      
  
  
  

In [0]:
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(44),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.TenCrop(44),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

In [0]:
trainset = FerDataset('train.csv', transforms=transform_train, train=True, valid=False)

In [0]:
import torch

In [0]:
trainloader = Data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=1)

In [0]:
testset = FerDataset('test.csv', transforms=transform_test, train=False, valid=False)

In [0]:
testloader = Data.DataLoader(testset, batch_size=256, shuffle=True, num_workers=1)

In [0]:
validset = FerDataset('validation.csv', transforms=transform_train, train=False, valid=True)
validloader = Data.DataLoader(validset, batch_size=256, shuffle=True, num_workers=1)

In [0]:
import torch.nn as nn
import torch.nn.functional as F


In [0]:
class model(nn.Module):
  
  def __init__(self):
    
    super(model, self).__init__()
    
    n_features = 48*48
    
    n_out = 7
    
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1)
    
    self.conv2 = nn.Conv2d(in_channels=16 , out_channels=32, kernel_size=3, stride=1)
    
    self.conv3 = nn.Conv2d(in_channels=32 , out_channels=64, kernel_size=3, stride=1)
    
    self.pool = nn.MaxPool2d(2, 2)
    
    self.fc = nn.Linear(n_features, 64)
    
    self.out = nn.Linear(64, n_out)
    
    self.dropout = nn.Dropout(0.2)
    
    
  def forward(self, x):
    
    x = F.relu(self.conv1(x))
    
    x = self.pool(x)
    
    x = F.relu(self.conv2(x))
    
    x = self.pool(x)
    
    x = F.relu(self.conv3(x))
    
    x = self.pool(x)
    
    x = x.view(-1, 48*48)
    
    x = self.dropout(x)
    
    x = F.relu(self.fc(x))
    
    x = self.dropout(x)
    
    x = self.out(x)
    
    return x
  
    
    
    

In [24]:
net = model()
print(net)

model(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=2304, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=7, bias=True)
  (dropout): Dropout(p=0.2)
)


In [25]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  print('Training on CPU...')
  
else:
  print('Training on GPU...')
    
if train_on_gpu:
  net.cuda()

Training on GPU...


In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.03)

In [27]:
# Training the network

valid_loss_min = np.Inf

for epoch in range(1, epochs + 1):
  
  train_loss = 0.0
  valid_loss = 0.0
  
  net.train()
  
  for img, target in trainloader:
    
    if train_on_gpu:
      img, target = img.cuda(), target.cuda()
      
    optimizer.zero_grad()
    
    output = net(img)
    
    loss = criterion(output, target)
    
    loss.backward()
    
    optimizer.step()
    
    train_loss += loss.item()*img.size(0)
    
    



torch.utils.data.dataloader.DataLoader